In [ ]:
!pip install underthesea
!pip install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.6 MB 7.7 MB/s 
     |████████████████████████████████| 235 kB 58.6 MB/s 
     |████████████████████████████████| 965 kB 20.0 MB/s 
     |████████████████████████████████| 581 kB 28.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import math
import numpy as np
import pandas as pd
import gensim
import string
import random
from underthesea import word_tokenize
import nltk
nltk.download('punkt')
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
dataset_chatbot_1 = pd.read_csv("/content/drive/MyDrive/chatbot data.csv")
dataset_chatbot_2 = pd.read_csv("/content/drive/MyDrive/dữ liệu chatbot question-answer short style.csv")

In [ ]:
dataset_chatbot_1.isin(['?']).sum(axis=0)

Unnamed: 0    0
user_a        0
user_b        0
dtype: int64

In [ ]:
dataset_chatbot_2.isin(['?']).sum(axis=0)

Unnamed: 0    0
user_a        2
user_b        2
dtype: int64

In [ ]:
dataset_chatbot_2['user_a'] = dataset_chatbot_2['user_a'].replace('?',np.nan)
dataset_chatbot_2['user_b'] = dataset_chatbot_2['user_b'].replace('?',np.nan)

In [ ]:
dataset_chatbot_1.dropna(inplace=True)
dataset_chatbot_2.dropna(inplace=True)

In [ ]:
def removePunctuations(sen):
    temp_l = sen.split()
    i = 0
    j = 0
    for word in temp_l :
        j = 0
        for l in word :
            if l in string.punctuation:
                word = word.replace(l," ")
            j += 1
        temp_l[i] = word.lower()
        i=i+1   
    content = " ".join(temp_l)
    return content

In [ ]:
question_tokens = []

for data in dataset_chatbot_1['user_a']:
    if "<eos>" in data.split():
        data = data.replace(" <eos> ",", ")
    question_tokens.append(word_tokenize(removePunctuations(data.lower())))

for data in dataset_chatbot_2['user_a']:
    question_tokens.append(word_tokenize(removePunctuations(data.lower())))

print(question_tokens[:5])
print(len(question_tokens))

[['bạn', 'tên', 'gì'], ['tui', 'tên', 'nam', 'bạn', 'học', 'ở', 'đâu'], ['tui', 'học', 'ở', 'tdtu', 'luôn', 'bạn', 'học', 'ngành', 'gì'], ['chuyên', 'ngành', 'gì', 'vậy', 'bạn'], ['mình', 'học', 'quản trị', 'kinh doanh', 'bạn', 'là', 'người', 'ở', 'đâu', 'vậy']]
7274


In [ ]:
ans = []
for data in dataset_chatbot_1['user_b']:
    if "<eos>" in data.split():
        data = data.replace(" <eos> ",", ")
    ans.append(data)

for data in dataset_chatbot_2['user_b']:
    ans.append(data)
print(ans[:5])
print(len(ans))

['Mình tên Trường, còn bạn tên gì? ', 'tui học ở TDTU nè, còn bạn học ở trường nào ?', 'mình học Công Nghệ Thông Tin', 'mình học bên Kỹ Thuật Phần Mềm á, còn bạn học ngành gì ?', 'Mình là người Đồng tháp, bạn biết Đồng Tháp không ?']
7274


In [ ]:
from gensim.models import Word2Vec
modelw2v = gensim.models.Word2Vec(sentences=question_tokens, min_count=1)

In [ ]:
def response_using_wm(sent,question=question_tokens,answer=ans):
    similarity = []
    sent = word_tokenize(removePunctuations(sent.lower()))
    for ques in question:
        similarity.append(modelw2v.wmdistance(sent, ques))
    similarity_des = np.argsort(similarity)
    return answer[similarity_des[0]]

In [ ]:
def response_using_euclid(sent,question=question_tokens,answer=ans):
    euc_dict = {}
    sent = word_tokenize(removePunctuations(sent.lower()))
    for index, ques in enumerate(question):
        if np.array(modelw2v.wv[sent]).shape ==  np.array(modelw2v.wv[ques]).shape:
            euc_dict[index] = np.linalg.norm( np.array(modelw2v.wv[sent]) -  np.array(modelw2v.wv[ques]))
    sort_euc = {k: v for k, v in sorted(euc_dict.items(), key=lambda item: item[1])}
    return answer[list(sort_euc.keys())[0]]

In [ ]:
sent = "Bạn đang làm gì ?"
sent = word_tokenize(removePunctuations(sent.lower()))
modelw2v.wv[sent]

array([[ 3.57329212e-02,  6.20594099e-02,  3.36179793e-01,
        -1.00329757e-01, -9.90750492e-02, -1.21671237e-01,
        -2.15358227e-01,  2.32738689e-01, -1.59754679e-01,
        -1.02083087e-01,  7.61567503e-02, -3.01979512e-01,
         2.14191660e-01,  1.47739677e-02, -3.37231040e-01,
         2.03178838e-01,  7.03935266e-01, -1.54051796e-01,
        -3.41760188e-01,  1.21638194e-01, -2.63899505e-01,
         7.97400326e-02,  5.06815128e-02,  4.95455880e-03,
         2.30406240e-01,  3.77387881e-01,  3.97910953e-01,
         2.54635841e-01, -3.94080542e-02,  1.98306724e-01,
         2.60371149e-01,  1.80918783e-01, -3.58853132e-01,
        -2.78038800e-01,  8.96883130e-01,  3.43754560e-01,
         1.27619470e-03,  5.00578701e-01, -4.94639665e-01,
         2.62692869e-01, -1.78577647e-01,  2.24137604e-01,
        -5.61072767e-01,  3.99546623e-01,  4.23646644e-02,
        -4.49087679e-01,  3.81990403e-01, -1.84664220e-01,
         2.43198588e-01, -3.95828813e-01,  2.29261398e-0

In [ ]:
def chat_with_bot(res=response_using_wm):
    GREETING_INPUTS = ["Chào Bot","hi","hey"]
    GREETING_RESPONSES = ["hi", "hey", "hello", "Tôi rất vui, khi bạn muốn trò chuyện với tôi"]
    def greeting(sentence):
        if sentence.lower() in [g.lower() for g in GREETING_INPUTS]:
            return random.choice(GREETING_RESPONSES)

    flag=True
    print("BOT: Tên tôi là Bot. Tôi sẽ trả lời các câu hỏi của bạn về Chatbots. Nếu bạn muốn thoát, hãy nhập Bye!")
    while(flag==True):
        user_response = input()
        user_response=user_response.lower()
        if(user_response!='bye'):
            if(user_response=='cảm ơn' or user_response=='thanks' ):
                flag=False
                print("BOT: Không có gì? Rất vui được trò chuyện với bạn.")
            else:
                if(greeting(user_response)!=None):
                    print("BOT: "+ greeting(user_response))
                else:
                    response_bot = "";
                    if('và' in user_response.split()):
                        user_res= user_response.split(" và ")
                        for i in range(len(user_res)):
                            if i == 0 or i % 2 == 0:
                                response_bot = response_bot + removePunctuations(res(sent = user_res[i]))
                            else:
                                response_bot = response_bot + ", " + removePunctuations(res(sent = user_res[i]))
                        print("BOT: ", end="")
                        print(response_bot)
                    else:
                        print("BOT: ", end="")
                        print(res(sent = user_response))
                    
        else:
            flag=False
            print("BOT: Bye!")

In [ ]:
chat_with_bot()

BOT: Tên tôi là Bot. Tôi sẽ trả lời các câu hỏi của bạn về Chatbots. Nếu bạn muốn thoát, hãy nhập Bye!
hi
BOT: hello
Bạn đang làm gì ?
BOT: Trả lời những câu hỏi của bạn
Nhà bạn ở đâu ?
BOT: dữ liệu cá nhân xin lỗi không thể tiết lộ
Bạn có thích đi du lịch không?
BOT: Có chứ, mình rất thích đi du lịch
Bạn thích đi nơi nào nhất?
BOT: Mình thích đi Nha Trang nhất
Tại sao bạn thích du lịch Nha Trang?
BOT: Vì bãi biển nơi đó rất đẹp và nhiều khu nghỉ dưỡng
bye
BOT: Bye!


In [ ]:
chat_with_bot(res=response_using_euclid)

BOT: Tên tôi là Bot. Tôi sẽ trả lời các câu hỏi của bạn về Chatbots. Nếu bạn muốn thoát, hãy nhập Bye!
Bạn quê ở đâu?
BOT: mình quê ở Tiền Giang
Cho mình hỏi quê bạn ở đâu?
BOT: bạn bè
bye
BOT: Bye!


**Tài liệu tham khảo**
<ul>
<li>https://radimrehurek.com/gensim/auto_examples/tutorials/run_word2vec.html#</li>
<li>https://radimrehurek.com/gensim/models/word2vec.html</li>
<li>https://stackoverflow.com/questions/57579009/how-to-change-the-code-to-find-the-euclidean-distance-not-cosine-between-words</li>
</ul>